In [ ]:
import pandas as pd

df_main = pd.read_csv("Motor_Vehicle_Collisions_-_Crashes_20250506.csv")
df_vehicles = pd.read_csv("Motor_Vehicle_Collisions_-_Vehicles_20250506.csv")

df_main = df_main[['COLLISION_ID', 'BOROUGH', 'LATITUDE', 'LONGITUDE']]
df_vehicles = df_vehicles[['COLLISION_ID', 'STATE_REGISTRATION']]

vehicles_grouped = (
    df_vehicles
    .groupby('COLLISION_ID')['STATE_REGISTRATION']
    .apply(lambda x: x.head(5).tolist())
    .reset_index()
)

# Expand the list into separate VEHICLE1..VEHICLE5 columns
vehicles_expanded = pd.DataFrame(vehicles_grouped['STATE_REGISTRATION'].tolist(),
                                 columns=[f'VEHICLE{i}' for i in range(1, 6)])
vehicles_expanded['COLLISION_ID'] = vehicles_grouped['COLLISION_ID']

# Merge with the main dataframe on COLLISION_ID
df_merged = pd.merge(df_main, vehicles_expanded, on='COLLISION_ID', how='left')

# Remove data without coordinates
before_drop = len(df_merged)
df_merged = df_merged.dropna(subset=['BOROUGH', 'LATITUDE', 'LONGITUDE', 'VEHICLE1'])
after_drop = len(df_merged)
print(f"Number of removed entries: {before_drop - after_drop}")


def infer_borough(lat, lon):
    if lat is None or lon is None:
        return None
    # Approximate bounding boxes for each borough
    if 40.49 <= lat <= 40.92 and -74.26 <= lon <= -73.7:
        if lat > 40.8:
            if lon > -73.85:
                return 'QUEENS'
            else:
                return 'BRONX'
        else:
            if lon > -73.95:
                return 'QUEENS'
            elif lon > -74.0:
                return 'BROOKLYN'
            elif lon > -74.15:
                return 'STATEN ISLAND'
            else:
                return 'MANHATTAN'
    return None

df_merged['BOROUGH'] = df_merged.apply(
    lambda row: infer_borough(row['LATITUDE'], row['LONGITUDE']) if pd.isna(row['BOROUGH']) else row['BOROUGH'],
    axis=1
)

vehicle_cols = ['VEHICLE1', 'VEHICLE2', 'VEHICLE3', 'VEHICLE4', 'VEHICLE5']
df_long = df_merged.melt(
    id_vars=['BOROUGH'],
    value_vars=vehicle_cols,
    value_name='STATE'
)

# Step 2: Clean the STATE column
df_long = df_long.dropna(subset=['STATE'])
df_long['STATE'] = df_long['STATE'].str.upper().str.strip()

# Step 3: Group by BOROUGH and STATE to count occurrences
df_quantity = (
    df_long.groupby(['BOROUGH', 'STATE'])
    .size()
    .reset_index(name='QUANTITY')
)

valid_us_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
}

df_quantity = df_quantity[df_quantity['STATE'].isin(valid_us_states)]

/tmp/ipykernel_3797/1279918629.py:3: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

/tmp/ipykernel_3797/1279918629.py:4: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



Number of removed entries: 832434


In [ ]:
import plotly.express as px

print(df_quantity)

# Turn the data into a choropleth visualization 
fig = px.choropleth(
    df_quantity,
    locations="STATE",            # This should be US state abbreviations
    locationmode="USA-states",    # Map type
    color="QUANTITY",        # This should be the value
    hover_name="STATE",           # Hover shows state name
    animation_frame="BOROUGH",    # Animate by NYC borough
    scope="usa",                  # USA only
    color_continuous_scale="Reds",
)

fig.write_html("us_states.html")

           BOROUGH STATE  QUANTITY
1            BRONX    AK        11
2            BRONX    AL       289
3            BRONX    AR        73
4            BRONX    AZ      1182
6            BRONX    CA       420
..             ...   ...       ...
353  STATEN ISLAND    VT        25
354  STATEN ISLAND    WA        23
355  STATEN ISLAND    WI        16
356  STATEN ISLAND    WV         9
357  STATEN ISLAND    WY         4

[244 rows x 3 columns]
